# Análisis de Transacciones - Datathon
## Unión de Datos Transaccionales y Demográficos

# Revisar si son Recurrentes los datos

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

# --- 0. Definir Rutas ---
PATH_TX = Path("../backend/data/transactions_clean.csv")          # Transacciones limpias (desde script Polars)
PATH_CLI = Path("../backend/data/base_clientes_final.csv")         # Clientes
PATH_OUT = Path("../backend/data/transacciones_para_modelo.parquet") # Salida para el modelo

# --- Funciones Auxiliares ---
def calculate_age_at_transaction(birth_date, transaction_date):
    """
    Calcula la edad (en años) en el momento de la transacción.
    """
    if pd.isna(birth_date) or pd.isna(transaction_date):
        return None
    if birth_date > transaction_date:
        return 0 # Handle anomaly: birth_date in future
    age = transaction_date.year - birth_date.year - \
          ((transaction_date.month, transaction_date.day) < (birth_date.month, birth_date.day))
    return age

def calculate_customer_seniority_at_transaction(signup_date, transaction_date):
    """
    Calcula la antigüedad del cliente (en años) en el momento de la transacción.
    """
    if pd.isna(signup_date) or pd.isna(transaction_date):
        return None
    if signup_date > transaction_date:
        return 0 # Handle anomaly: signup_date in future
    seniority_days = (transaction_date - signup_date).days
    return seniority_days / 365.25

def main_data_preparation():
    print("=== INICIANDO PREPARACIÓN DE DATOS CON PANDAS ===")

    # --- 1. Cargar los datos ---
    try:
        df_clientes = pd.read_csv(PATH_CLI)
        df_transacciones = pd.read_csv(PATH_TX)
    except FileNotFoundError as e:
        print(f"Error: No se encontró el archivo {e.filename}. Verifica las rutas: \nClientes: {PATH_CLI}\nTransacciones: {PATH_TX}")
        return None
    except Exception as e:
        print(f"Ocurrió un error al cargar los archivos CSV: {e}")
        return None

    print("Archivos CSV cargados exitosamente.")

    # --- 2. Preprocesamiento y Conversión de Tipos ---
    # Clientes
    df_clientes['fecha_nacimiento'] = pd.to_datetime(df_clientes['fecha_nacimiento'], errors='coerce')
    df_clientes['fecha_alta'] = pd.to_datetime(df_clientes['fecha_alta'], errors='coerce')
    df_clientes['id'] = df_clientes['id'].astype(str)
    df_clientes['genero'] = df_clientes['genero'].fillna('Desconocido')
    df_clientes['tipo_persona'] = df_clientes['tipo_persona'].fillna('Desconocido')

    # Transacciones
    df_transacciones['fecha'] = pd.to_datetime(df_transacciones['fecha'], errors='coerce')
    df_transacciones['id'] = df_transacciones['id'].astype(str)
    df_transacciones['monto'] = pd.to_numeric(df_transacciones['monto'], errors='coerce')

    df_clientes.dropna(subset=['id', 'fecha_nacimiento', 'fecha_alta'], inplace=True)
    df_transacciones.dropna(subset=['id', 'fecha', 'monto'], inplace=True)

    print("\nPreprocesamiento y conversión de tipos completados.")

    # --- 3. Unir DataFrames ---
    cols_clientes_a_usar = ['id', 'fecha_nacimiento', 'fecha_alta', 'genero', 'tipo_persona']
    df_merged = pd.merge(df_transacciones, df_clientes[cols_clientes_a_usar], on='id', how='left')

    df_merged.dropna(subset=['fecha_nacimiento', 'fecha_alta'], inplace=True)
    if df_merged.empty:
        print("Error: El DataFrame fusionado está vacío después de eliminar filas sin 'fecha_nacimiento' o 'fecha_alta'.")
        return None

    print(f"\nDataFrame fusionado. Total de filas: {len(df_merged)}")

    # --- 4. Ingeniería de Características ---
    df_merged['edad_transaccion'] = df_merged.apply(
        lambda row: calculate_age_at_transaction(row['fecha_nacimiento'], row['fecha']), axis=1
    ).astype(float)

    df_merged['antiguedad_cliente'] = df_merged.apply(
        lambda row: calculate_customer_seniority_at_transaction(row['fecha_alta'], row['fecha']), axis=1
    ).astype(float)

    # Rellenar NaNs resultantes de funciones calculate si es necesario (ej. si alguna fecha fue NaT)
    df_merged['edad_transaccion'].fillna(df_merged['edad_transaccion'].median(), inplace=True)
    df_merged['antiguedad_cliente'].fillna(df_merged['antiguedad_cliente'].median(), inplace=True)


    # Ordenar los datos para cálculos de recurrencia basados en series temporales por cliente y comercio
    df_merged.sort_values(['id', 'comercio', 'fecha'], inplace=True)
    print("\nDataFrame ordenado por id, comercio, fecha.")

    # Características de recurrencia (usando shift y expanding para evitar look-ahead bias)
    # `dias_desde_ultima_compra_comercio`
    df_merged['lag_fecha_comercio'] = df_merged.groupby(['id', 'comercio'])['fecha'].shift(1)
    df_merged['dias_desde_ultima_compra_comercio'] = (df_merged['fecha'] - df_merged['lag_fecha_comercio']).dt.days.fillna(9999).astype(int) # Usar un valor grande para la primera compra

    # `num_transacciones_previas_comercio`
    df_merged['num_transacciones_previas_comercio'] = df_merged.groupby(['id', 'comercio']).cumcount()

    # `monto_promedio_comercio` (promedio de montos de transacciones *previas* en el mismo comercio)
    df_merged['monto_promedio_comercio'] = df_merged.groupby(['id', 'comercio'])['monto'].transform(lambda x: x.expanding().mean().shift(1)).fillna(0).astype(float)
    # ^ Se usa shift(1) en expanding() para asegurar que solo se usan datos *previos*

    # `std_dias_entre_compras` (std de los días entre compras *previas* en el mismo comercio)
    df_merged['dias_entre_esta_y_anterior_comercio'] = df_merged.groupby(['id', 'comercio'])['fecha'].diff().dt.days
    df_merged['std_dias_entre_compras'] = df_merged.groupby(['id', 'comercio'])['dias_entre_esta_y_anterior_comercio'].transform(lambda x: x.expanding().std().shift(1)).fillna(0).astype(float)

    # `diff_monto_promedio` (diferencia relativa entre el monto actual y el promedio previo)
    df_merged['diff_monto_promedio'] = np.abs(df_merged['monto'] - df_merged['monto_promedio_comercio']) / df_merged['monto_promedio_comercio']
    # Manejar división por cero/infinito si monto_promedio_comercio es 0 o NaN
    df_merged['diff_monto_promedio'].replace([np.inf, -np.inf], np.nan, inplace=True)
    df_merged['diff_monto_promedio'].fillna(0, inplace=True) # Rellenar con 0 si no hay promedio previo o es 0

    # `monto_similar` (boolean)
    df_merged['monto_similar'] = (df_merged['diff_monto_promedio'] <= 0.2).astype(bool)


    # --- NUEVA DEFINICIÓN DE 'es_recurrente' (Para que el modelo APRENDA) ---
    # La variable objetivo 'es_recurrente' ahora indica si la transacción exhibe
    # un patrón de frecuencia y/o consistencia de monto que sugiera recurrencia,
    # sin ser una definición perfecta que cause data leakage trivial.
    # El modelo Random Forest intentará aprender esta lógica de forma más compleja.

    # Inicializa 'es_recurrente' como False
    df_merged['es_recurrente'] = False

    # Condición principal: No es la primera transacción para ese cliente/comercio
    # y hay un patrón de repetición o consistencia.
    # Esto es crucial para asegurar que la primera compra NUNCA es recurrente.
    base_condition_not_first_purchase = (df_merged['num_transacciones_previas_comercio'] >= 1)

    # Regla 1: Frecuencia y consistencia temporal (similar a una suscripción)
    # Buscamos transacciones que se repiten en intervalos "razonables" y de forma consistente.
    condition_frequent_consistent = (
        (df_merged['dias_desde_ultima_compra_comercio'].between(1, 45)) & # Entre 1 día y 45 días
        (df_merged['std_dias_entre_compras'] <= 15) & # Baja variabilidad en los intervalos de compra
        (df_merged['num_transacciones_previas_comercio'] >= 2) # Al menos 2 previas (tercera compra en adelante)
    )
    df_merged.loc[base_condition_not_first_purchase & condition_frequent_consistent, 'es_recurrente'] = True

    # Regla 2: Agregadores o comercios de "gasto frecuente" con montos similares
    # Esto captura giros donde se espera mucha frecuencia, y el monto es parecido.
    GIROS_GASTO_FRECUENTE = [
        "COMERCIOS ELECTRONICOS (VTAS POR INTERNET)",
        "SERVICIOS DE STREAMING",
        "TELECOMUNICACIONES",
        "RESTAURANTES", # Si consideras restaurantes frecuentes para algunos clientes
        "CAFETERIAS"
    ] # Puedes expandir esta lista con giros que sean recurrentes en tu negocio

    condition_giro_frequent_similar_monto = (
        df_merged['giro_comercio'].isin(GIROS_GASTO_FRECUENTE) &
        (df_merged['dias_desde_ultima_compra_comercio'].between(1, 60)) & # Un rango más amplio de días
        (df_merged['monto_similar']) & # El monto es similar al promedio
        (df_merged['num_transacciones_previas_comercio'] >= 1) # Ya hubo al menos una previa
    )
    df_merged.loc[base_condition_not_first_purchase & condition_giro_frequent_similar_monto, 'es_recurrente'] = True

    # Regla 3: Compras muy frecuentes, aunque no sean "suscripciones" típicas
    # Captura patrones donde la gente compra a menudo del mismo lugar (ej. supermercado semanal).
    condition_very_frequent = (
        (df_merged['dias_desde_ultima_compra_comercio'].between(1, 30)) &
        (df_merged['num_transacciones_previas_comercio'] >= 4) # Al menos 5 compras en total
    )
    df_merged.loc[base_condition_not_first_purchase & condition_very_frequent, 'es_recurrente'] = True

    # Asegurarse de que la primera transacción con un comercio nunca sea recurrente
    # (aunque base_condition_not_first_purchase ya se encarga de esto)
    df_merged.loc[df_merged['num_transacciones_previas_comercio'] == 0, 'es_recurrente'] = False

    df_merged['es_recurrente'] = df_merged['es_recurrente'].astype(bool)

    print("\nIngeniería de características completada.")
    print("Distribución de la variable objetivo 'es_recurrente' (con criterios ajustados):")
    print(df_merged['es_recurrente'].value_counts(normalize=True))


    # --- 5. Seleccionar y Ordenar Columnas Finales ---
    columnas_finales = [
        "id", "fecha", "comercio", "giro_comercio", "tipo_venta", "monto",
        "edad_transaccion", "genero", "tipo_persona", "antiguedad_cliente",
        "dias_desde_ultima_compra_comercio", "num_transacciones_previas_comercio", # Incluida como feature
        "monto_promedio_comercio", "std_dias_entre_compras",
        "diff_monto_promedio", "monto_similar", # Incluir estas como features para el modelo
        "es_recurrente" # La variable objetivo (con criterios ajustados)
    ]

    df_final = df_merged[columnas_finales].copy()

    # Asegurar los tipos de datos finales
    df_final['id'] = df_final['id'].astype(str)
    df_final['fecha'] = pd.to_datetime(df_final['fecha'])
    df_final['comercio'] = df_final['comercio'].astype(str)
    df_final['giro_comercio'] = df_final['giro_comercio'].astype(str)
    df_final['tipo_venta'] = df_final['tipo_venta'].astype(str)
    df_final['monto'] = df_final['monto'].astype(float)
    df_final['edad_transaccion'] = df_final['edad_transaccion'].astype(float)
    df_final['genero'] = df_final['genero'].astype(str)
    df_final['tipo_persona'] = df_final['tipo_persona'].astype(str)
    df_final['antiguedad_cliente'] = df_final['antiguedad_cliente'].astype(float)
    df_final['dias_desde_ultima_compra_comercio'] = df_final['dias_desde_ultima_compra_comercio'].astype(int)
    df_final['num_transacciones_previas_comercio'] = df_final['num_transacciones_previas_comercio'].astype(int)
    df_final['monto_promedio_comercio'] = df_final['monto_promedio_comercio'].astype(float)
    df_final['std_dias_entre_compras'] = df_final['std_dias_entre_compras'].astype(float)
    df_final['diff_monto_promedio'] = df_final['diff_monto_promedio'].astype(float)
    df_final['monto_similar'] = df_final['monto_similar'].astype(bool)
    df_final['es_recurrente'] = df_final['es_recurrente'].astype(bool)

    print("\nColumnas finales seleccionadas y ordenadas.")

    # --- 6. Guardar como Parquet ---
    try:
        PATH_OUT.parent.mkdir(parents=True, exist_ok=True)
        df_final.to_parquet(PATH_OUT, index=False, engine='pyarrow')
        print(f"\nDataFrame guardado exitosamente como '{PATH_OUT}'")
    except ImportError:
        print("\nError: Necesitas instalar 'pyarrow' para guardar en formato Parquet.")
        print("Puedes instalarlo con: pip install pyarrow")
    except Exception as e:
        print(f"\nOcurrió un error al guardar el archivo Parquet: {e}")

    return df_final

def main_model():
    print("\n=== INICIANDO FASE DE MODELADO CON SCIKIT-LEARN ===")

    df = None
    try:
        df = pd.read_parquet(PATH_OUT)
        print(f"Datos cargados exitosamente desde: {PATH_OUT}")
        print(f"Dimensiones del DataFrame: {df.shape}")
        print("Primeras filas:")
        print(df.head())
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo en {PATH_OUT}. Asegúrate de que la etapa de preparación de datos se haya ejecutado correctamente.")
        return
    except Exception as e:
        print(f"Ocurrió un error al cargar el archivo Parquet para el modelo: {e}")
        return

    TARGET_COLUMN = 'es_recurrente'

    print(f"\n--- Análisis Exploratorio Básico de '{TARGET_COLUMN}' ---")
    if TARGET_COLUMN not in df.columns:
        print(f"Error: La columna objetivo '{TARGET_COLUMN}' no se encuentra en el DataFrame.")
        return

    print(f"Distribución de la variable objetivo '{TARGET_COLUMN}':")
    print(df[TARGET_COLUMN].value_counts(normalize=True))

    print("\nInformación del DataFrame:")
    df.info()

    numerical_cols_with_na = df[df.select_dtypes(include=np.number).columns].isnull().sum()
    numerical_cols_with_na = numerical_cols_with_na[numerical_cols_with_na > 0].index.tolist()
    if numerical_cols_with_na:
        print(f"\nDetectados NaNs en columnas numéricas: {numerical_cols_with_na}. Rellenando con la mediana.")
        for col in numerical_cols_with_na:
            df[col].fillna(df[col].median(), inplace=True)

    categorical_cols_with_na = df[df.select_dtypes(exclude=np.number).columns].isnull().sum()
    categorical_cols_with_na = categorical_cols_with_na[categorical_cols_with_na > 0].index.tolist()
    if categorical_cols_with_na:
        print(f"\nDetectados NaNs en columnas categóricas: {categorical_cols_with_na}. Rellenando con la moda.")
        for col in categorical_cols_with_na:
            df[col].fillna(df[col].mode()[0], inplace=True)


    # --- 8. Selección de Características (Features) y Variable Objetivo (Target) ---
    features = [
        "comercio", "giro_comercio", "tipo_venta", "monto",
        "edad_transaccion", "genero", "tipo_persona", "antiguedad_cliente",
        "dias_desde_ultima_compra_comercio", "num_transacciones_previas_comercio",
        "monto_promedio_comercio", "std_dias_entre_compras",
        "diff_monto_promedio", "monto_similar"
    ]

    features = [f for f in features if f in df.columns]

    X = df[features]
    y = df[TARGET_COLUMN].astype(int)

    print(f"\n--- Preparación de Datos para el Modelo ---")
    print(f"Dimensiones de X (features): {X.shape}")
    print(f"Dimensiones de y (target): {y.shape}")

    # --- 9. Preprocesamiento de Características (para Sklearn Pipeline) ---
    numerical_features = X.select_dtypes(include=np.number).columns.tolist()
    categorical_features = X.select_dtypes(exclude=np.number).columns.tolist()

    print(f"\nCaracterísticas numéricas identificadas: {numerical_features}")
    print(f"Características categóricas identificadas: {categorical_features}")

    numerical_transformer = StandardScaler()
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_features),
            ('cat', categorical_transformer, categorical_features)
        ],
        remainder='passthrough'
    )

    # --- 10. División de Datos en Entrenamiento y Prueba ---
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=0.25, # 25% para prueba, 75% para entrenamiento
        random_state=42,
        stratify=y
    )

    print(f"\nDimensiones de X_train: {X_train.shape}, y_train: {y_train.shape}")
    print(f"Dimensiones de X_test: {X_test.shape}, y_test: {y_test.shape}")
    print(f"Proporción de '{TARGET_COLUMN}' en y_train:\n{y_train.value_counts(normalize=True)}")
    print(f"Proporción de '{TARGET_COLUMN}' en y_test:\n{y_test.value_counts(normalize=True)}")

    # --- 11. Creación y Entrenamiento del Pipeline del Modelo ---
    model = RandomForestClassifier(
        random_state=42,
        n_estimators=200,
        class_weight='balanced', # Sigue siendo útil dado el desbalance
        max_depth=10,
        min_samples_leaf=5
    )

    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                               ('classifier', model)])

    print("\n--- Entrenando el Modelo Random Forest para Recurrencia ---")
    pipeline.fit(X_train, y_train)
    print("Modelo entrenado exitosamente.")

    import joblib
    joblib.dump(pipeline, 'modelo_recurrencia.pkl')
    print("Modelo de recurrencia guardado exitosamente.")

    # --- 12. Evaluación del Modelo ---
    print("\n--- Evaluación del Modelo en el Conjunto de Prueba ---")
    y_pred = pipeline.predict(X_test)
    y_pred_proba = pipeline.predict_proba(X_test)[:, 1]

    print("\nReporte de Clasificación:")
    print(classification_report(y_test, y_pred))

    print("\nMatriz de Confusión:")
    cm = confusion_matrix(y_test, y_pred)
    print(cm)

    try:
        auc_score = roc_auc_score(y_test, y_pred_proba)
        print(f"\nAUC-ROC Score: {auc_score:.4f}")
    except ValueError as e:
        print(f"\nNo se pudo calcular AUC-ROC: {e}. Esto puede pasar si solo una clase está presente en y_true.")

    # --- 12.1. Importancia de las Características ---
    print("\n--- Importancia de las Características del Modelo ---")
    try:
        numerical_feature_names_transformed = numerical_features
        # Get feature names after one-hot encoding for categorical features
        categorical_feature_names_transformed = pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names_out(categorical_features)
        all_feature_names = list(numerical_feature_names_transformed) + list(categorical_feature_names_transformed)
        importances = pipeline.named_steps['classifier'].feature_importances_
        sorted_importances = sorted(zip(importances, all_feature_names), reverse=True)
        for i, (imp, name) in enumerate(sorted_importances[:15]): # Mostrar las 15 más importantes
            print(f"{i+1}. {name}: {imp:.4f}")
    except Exception as e:
        print(f"No se pudo obtener la importancia de las características: {e}")
        print("Asegúrate de que 'classifier' es un modelo con attribute 'feature_importances_'.")


    # --- 13. Próximos Pasos para tu Objetivo Final ---
    print("\n--- Próximos Pasos Sugeridos para Predecir CUÁNTO y CUÁNDO ---")
    print("1. **Re-evaluar el Modelo Clasificador**: Con esta nueva definición, las métricas serán más realistas y el modelo aprenderá de forma más robusta.")
    print("2. **Identificar Transacciones Recurrentes**: Usa el modelo clasificador para predecir 'es_recurrente' en nuevos datos.")
    print("3. **Preparar Datos para Modelos de Regresión**: Filtra tu DataFrame original para incluir SOLO las transacciones que el clasificador identificó como recurrentes.")
    print("4. **Ingeniería de Características para Regresión**: Crea features específicas para predecir monto y fecha (ej. tendencias de monto, estacionalidad, etc.).")
    print("5. **Modelo de Regresión para 'Monto'**: Entrena un modelo (ej. RandomForestRegressor, LightGBM Regressor) para predecir el 'monto' de la próxima transacción recurrente.")
    print("6. **Modelo de Regresión/Series de Tiempo para 'Cuándo'**: Entrena un modelo para predecir la 'fecha' o 'días hasta la próxima transacción recurrente'. Esto podría ser más complejo, quizás usando modelos de series de tiempo o regresores que predigan un intervalo.")
    print("7. **Iteración**: Refina tus modelos, ajusta hiperparámetros y busca más características para mejorar la precisión.")


if __name__ == '__main__':
    df_prepared = main_data_preparation()

    if df_prepared is not None:
        main_model()

=== INICIANDO PREPARACIÓN DE DATOS CON PANDAS ===
Archivos CSV cargados exitosamente.

Preprocesamiento y conversión de tipos completados.

DataFrame fusionado. Total de filas: 346011


/var/folders/rd/d_mznhv14n19s_4d9_h190840000gn/T/ipykernel_55439/3199455379.py:96: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged['edad_transaccion'].fillna(df_merged['edad_transaccion'].median(), inplace=True)
/var/folders/rd/d_mznhv14n19s_4d9_h190840000gn/T/ipykernel_55439/3199455379.py:97: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediat


DataFrame ordenado por id, comercio, fecha.


/var/folders/rd/d_mznhv14n19s_4d9_h190840000gn/T/ipykernel_55439/3199455379.py:123: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged['diff_monto_promedio'].replace([np.inf, -np.inf], np.nan, inplace=True)
/var/folders/rd/d_mznhv14n19s_4d9_h190840000gn/T/ipykernel_55439/3199455379.py:124: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate objec


Ingeniería de características completada.
Distribución de la variable objetivo 'es_recurrente' (con criterios ajustados):
es_recurrente
True     0.616995
False    0.383005
Name: proportion, dtype: float64

Columnas finales seleccionadas y ordenadas.

DataFrame guardado exitosamente como '../data/transacciones_para_modelo.parquet'

=== INICIANDO FASE DE MODELADO CON SCIKIT-LEARN ===
Datos cargados exitosamente desde: ../data/transacciones_para_modelo.parquet
Dimensiones del DataFrame: (346011, 17)
Primeras filas:
                                         id      fecha comercio  \
0  003d9abe467a91847d566cf455bd2d7d6c8f7e75 2022-01-17   AMAZON   
1  003d9abe467a91847d566cf455bd2d7d6c8f7e75 2022-01-17   AMAZON   
2  003d9abe467a91847d566cf455bd2d7d6c8f7e75 2022-01-17   AMAZON   
3  003d9abe467a91847d566cf455bd2d7d6c8f7e75 2022-01-17   AMAZON   
4  003d9abe467a91847d566cf455bd2d7d6c8f7e75 2022-01-17   AMAZON   

                                giro_comercio tipo_venta  monto  \
0  COMERCIO